<h1><center>Cloud Computing  et informatique distribuée</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Exercises: TD on Map Reduce and Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# First Part : Introduction to MapReduce - the origins in Hadoop

MapReduce is a programming model useful in for big data processing and allows running a parallel and distributed  algorithm on a cluster.

A MapReduce algorithm is composed of:
<ul>
    <li> a map function, which performs a first set of operations on the input data and, usually, produces a set of key-value pairs as output  </li>
    <li> a reduce function, which performs an operation on the data coming as output from the map function grouped on the basis of the key.</li>
</ul>

The MapReduce algorithm can be run on a framework (i.e. Hadoop). In this case the framework will handle the process of
running the algorithm on the distributed architecture running the various tasks in parallel, managing all communications and data transfers between the various parts of the system, and providing for redundancy and fault tolerance. The map function and the reduce function will then become independent procedures running in multiple instances in parallel.

The map and the reduce procedures can be seen then as two main functions: map() et reduce().

### Map

In general the map() function is seen as a function that:
<ul>
    <li> takes as input one (or a series) key/value pair(s);  </li>
    <li> processes each key/value pair according to the procedure it implements;</li>
    <li> generates as output zero or more key/value pairs.</li>
</ul>

It is important to notice that:
<ul>
    <li> if the input data is composed by single values each sigle value can be seen as a key/value pair having an empty  value;  </li>
    <li> the types of the input key/value pair and the output key/value pair(s) can be (and often are) different from each other;</li>
    <li> output key/value pair can have a dummy key;</li>
    <li> also the output key/value pair can have an empty value.</li>
</ul>

In the "count words" of the Spark introduction exercises we counting the number of occurrences of each word in the provided file. The map function would take as input a single line. This single line can be seen as a key/value pair having as key the whole line and as value an empty value. 

After this phase another map function will break the line into words and output a key/value pair for each word (after the phase that filters the stop-words and the non-letters). 

The following reduce step will consider as (key,value) pair a string representing the word and integer equal to 1 as value. 


### Shuffle
Between the map() and reduce() functions, the data are shuffled in order to move together data sharing the same key.
Data will be then processed by the reduce function. 
When you run a MapReduce procedure on a framework (i.e. Hadoop) using the available primitives the shuffle operation is fully and transparently handled by the framework. When you use Spark then you do not have to think explicitely about the shuffle operation when you run reduce-like operations over a key.




### Reduce
The reduce() function applies a procedure on all the pairs that share the same key.  

It is important to notice that:
<ul>
    <li> the reduce function produces zero or more outputs for each group of key-value pairs  </li>
    <li> the type of the output can be different from the input and can be also one or a set of key/value pair(s);</li>
    <li> even in this case the key/value pair can have an empty value.</li>
</ul>



In the word count example, the reduce function takes the input values, sums them, and generates a single output containing a set of key/value pairs having as key the value of each word and as value the final sum.






In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar zxvf spark-3.2.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"


import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")

import numpy as np
import random as rn

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
spark-3.2.4-bin-hadoop2.7/
spark-3.2.4-bin-hadoop2.7/R/
spark-3.2.4-bin-hadoop2.7/R/lib/
spark-3.2.4-bin-hadoop2.7/R/lib/sparkr.zip
spark-3.2.4-bin-hadoop2.7/R/lib/SparkR/
spark-3.2.4-bin-hadoop2.7/R

# MapReduce in Spark

Remember that Spark engine providse a <code>map</code> operation on a RDD, a <code>reduce</code> operation on an RDD and a <code>reduceByKey</code> operation on a RDD.

The map and the reduce operations in Spark allow to implement the MapReduce programming paradighm with some little differences to take into account while programming.

### <code> Map</code>

<li> the concept of the key is relaxed;</li>
    <li>the <code>map</code> operations applies a function to all the elements of an RDD.</li>
</li>


In [ ]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())
[('a', 2), ('b', 1)]

### <code> Reduce </code>


<li> the concept of the key is relaxed;</li>
    <li>the <code>reduce</code> operation applies a commutative and associative binary operator to all the elements of an RDD.</li>
</li>

In [ ]:
sc.parallelize([1, 2, 3, 4, 5]).reduce(add)
15

### <code> ReduceByKey </code>


<li> the concept of the key is relaxed;</li>
    <li>the <code>reduceByKey</code> operation applies an associative and commutative reduce function merging all the elements of an RDD using a key.</li>
</li>

In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

 # Some utility functions for the next exercises

In [ ]:
def load_text(filename):
    return sc.textFile(filename)

mobydick = load_text('moby-dick.txt')


import re

# Regular expression for removing all non-letter characters in the file.
regex = re.compile('[^a-zA-Z ]')

'''
Removes any non-letter character from the given word.

INPUT:
        word: A word

OUTPUT:
        the input word without the non-letter characters.

'''
def remove_non_letters(word):
    return regex.sub('', word)


'''
INPUT: 
        text: RDD where each element is a line of the input text file.
        stopwords: Python list containing the stopwords.
OUTPUT: 
        RDD where each element is a word from the input text file.
'''
def preprocess(text, stopwords):
    words = text.flatMap(lambda line: line.split(" "))\
                .map(lambda word: remove_non_letters(word))\
                .filter(lambda word: len(word) > 0)\
                .map(lambda word: word.lower())\
                .filter(lambda word: word not in stopwords)
    return words
    
'''
INPUT: 
        stopwords_file: name of the file containing the stopwords.
OUTPUT:
        a Python list with the stopwords read from the file.
'''
def load_stopwords(stopwords_file):
    stopwords = []
    with open(stopwords_file) as file:
        for sw in file:
            stopwords.append(sw.strip())
    return stopwords

stopwords = load_stopwords("stopwords.txt")
words = preprocess(mobydick, stopwords)
words.takeOrdered(10, key = lambda x: x)

################# EXPECTED OUTPUT #################
#
# ['aback',
# 'aback',
# 'abaft',
# 'abaft',
# 'abandon',
# 'abandon',
# 'abandon',
# 'abandoned',
# 'abandoned',
# 'abandoned']
#
###################################################

In [ ]:
'''
INPUT:
        words: RDD, where each element is word from the input text file (preprocessing already done!).
OUTPUT:
        RDD, where each element is (w, occ), w is a word and occ the number of occurrences of w.
        The RDD is sorted by value in decreasing order.
'''
def word_count(words):    
    occs = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda x, y: x+y)\
                .sortBy(lambda f: f[1], ascending=False)
    return occs
    
occs = word_count(words)
occs.take(5)

################# EXPECTED OUTPUT #################
#
# [('whale', 891), ('one', 875), ('old', 436), ('man', 433), ('ahab', 417)]
#
###################################################

# Example: Counting global number of integer elements using Spark

<p align="justify">
<font size="3">
We start generalizing the strict pattern of MapReduce and we want to start taking advance of the Spark functions.

In this context, we take the mapReduce programming pattern as reference but we simplify the keys and we want to write the Spark code that counts the number of strings representing positive integers.

We take as input  a text file containing also strings representing floats.
    
We can write a possible solution running acconding to the following steps:

<ul>
    <li> a map function filtering the strings and giving as output 1 for the integers elements  and 0 for the non integer elements. </li>
    <li> a reduce function that performs the sum using the results coming from the map step.</li>
</ul>
    


<p align="justify">
<font size="3">
As first we write the code of the function $int\_map$ that performs the pre-process a text file splitting the strings when a space char is present.
    
The function has the following signature:
<ul>
<li> **Input.** A RDD $numbers$, where each element is a string from a text file.
<li> **Output.** A RDD, where each element is $(i)$, is equal to a string
</ul>
</font>
</p>


<hr style="border:solid 2px;">


For example the content of the file can be:

1 2 3.2 3 1.2 1.5

4 5 1.1 6 7 5.4

8 9 5.1 1 2 3 4.1

In [ ]:
'''
creating the RDD from the text file

'''
rdd_from_file = sc.textFile("numbers.txt")


'''
Parsing the lines of the RDD and splitting the strings

INPUT:
        an RDD

OUTPUT:
        an RDD of separate strings

'''

def int_map(numbers):
    return numbers.flatMap(lambda line: line.split(" "))

rdd_split_numbers = int_map(rdd_from_file)

print(rdd_split_numbers.take(5))


Then we can proceed and write a possible solution for the integer count running acconding to the following steps:

<ul>
    <li> writing a map function filtering the single chars and giving as output 1 for the integers elements  and 0 for the non integer elements. Notice that in this step we are not including the key migrating to the Spark more flexible pattern.</li>
    <li> a reduce function that is implemented by a sum that counts partial results coming from the map (using the dummy key) and returns the count (12 for this example).</li>
</ul>

In [ ]:
'''
The function checking the value of the string and mapping each string to 1 or 0:
1 if the string is the representation of an integer
0 otherwise. 

Notice that we are simplifying the map-reduce pattern: the output is not composed by a (key-value) pair since keeping trace of the origin
number value is not important for the final result.
INPUT:
        a string

OUTPUT:
        1 if the string is an integer (a string that does not contain any "non numeric char", 0 otherwise
        Notice that:
        "-1" is not a positive integer and the string contains "- that is not a numeric value"
        "1,1" or "1.1" are not positive integer and the strings contain "," and "." respectively that are not numeric values

'''

def map_int_strings_to_1(numbers):
    return numbers.map(lambda x : 1 if (x.isnumeric()) else 0)

'''
The function is performing a reduce just running a count that simply adds the list of numbers produced in the previous step
INPUT:
        an RDD having numbers as values

OUTPUT:
        the sum

'''

def reduce_sum(numbers_rdd):
    return numbers_rdd.sum()


print(reduce_sum(map_int_strings_to_1(rdd_split_numbers)))

################# EXPECTED OUTPUT #################
#
#  12 
#
###################################################
      

Of course it is not necessary to define different functions for the phases and you can write directly:

In [ ]:
result = rdd_split_numbers.map(lambda x : 1 if (x.isnumeric()) else 0).sum()

print(result)

################# EXPECTED OUTPUT #################
#
#  12 
#
###################################################
      

In this first series of exercises we will learn how to think programs according to the MapReduce paradigm.


# 1 Numbers


<p align="justify">
<font size="3">
Suppose again to use the same file where lines are still made of integers. 
</font>
</p>

    
## 1.1 Counting odd numbers and even numbers

<p align="justify">
<font size="3">
We must find how many even numbers and how many odd numbers are present the file following a MapReduce procedure. 

We would like to continue to generalize the procedure not including the keys but in this case we realize that maybe having a <code>map</code> function that produces a key-value pair can be useful for the reduce function that we apply after even in Spark context.
    
<p align="justify">
<font size="3">
We want to write the code of the function $even\_odd\_count$ that counts the number of occurrences 
The function has the following input and output:
<ul>
<li> **Input.** A RDD $numbers$, where each element is a number.
<li> **Output.** A RDD, with two elements:  $('o', v)$, $v$ being the number of odd numbers and $('e', v)$ being the number of even numbers.
</ul>
</font>
</p>





In [ ]:
'''
creating the RDD from the integer text file

'''
rdd_split_numbers = int_map(sc.textFile("integer_numbers.txt")) 
'''
The function even_odd_count

1) checks the value of an RDD of strings representing numbers and mapping each string to ("e", 1) or ("o", 1) and
according to the number they represent. 
("e", 1) if the string is the representation an even number
("o", 1) if the string is the representation an odd number

2) reduces and counts the (key-value pairs )
INPUT:
        an RDD of string values

OUTPUT:
        an RDD [('o', c_o), ('e', c_e)]

'''


'''############## WRITE YOUR CODE HERE ##############'''
def even_odd_count(numbers_rdd):
    numbers = 
    return numbers
'''############## END OF THE EXERCISE ##############'''


print(even_odd_count(rdd_split_numbers).collect())


################# EXPECTED OUTPUT #################
#
#  [('o', 14), ('e', 5)]
#
###################################################


## 1.2 Counting the occurrences of each number

<p align="justify">
<font size="3">
Suppose now that we want to count the number of occurrences of each number.
</font>
</p>



<p align="justify">
<font size="3">
We can go again step by step:



<ol>
<li> Define a function that produces pairs: </li>
    [(1, 1), (2, 1), ...]

<li>  think that data will be grouped together by a shuffle according to the key </li>
    [[[1, 1]], [[2, 1]], [[3, 1], [3, 1]], [[4, 1], [4, 1]], ...]

<li> apply a reduce that sums up the results </li>

</ol>
</font>
</p>




In [ ]:
'''
The function numbers_count

1) checks the value of an RDD of strings representing numbers and mapping each string to ("n", 1)
according to the number they represent. 

2) reduces and counts the (key-value pairs )
INPUT:
        an RDD of string values

OUTPUT:
        an RDD [('n_i', c_ni), ...)]

'''

'''############## WRITE YOUR CODE HERE ##############'''

def numbers_count(numbers_rdd):
    numbers= 
    return numbers

'''############## END OF THE EXERCISE ##############'''


print(numbers_count(rdd_split_numbers).collect())


################# EXPECTED OUTPUT #################
##  [('1', 2), ('1.5', 1), ('4', 1), ('5.4', 1), ('8', 1), ('9', 1), ('2', 2), ('3.2', 1), ('3', 2), ('1.2', 1), ('5', 1), ('1.1', 1), ('6', 1), ('7', 1), ('5.1', 1), ('4.1', 1)]
###################################################

Notice that the map-reduce procedure applied for the word count and the number count is the same. Once you learn thinking in map-reduce philosophy you can easily re-use your patterns.

# 2 Bigrams

<p align="justify">
<font size="3">
What is a Bigram? In our example a bigram is a couple of two consecutive words in a same line. 
    

For example, the previous sentence contains the following bigrams: "A bigram", "bigram is", "is a", "a couple", etc.
</font>
</p>

<p align="justify">
<font size="3">
This exercise can be seen as a simple extension of the word count, take into account this while implementing your data structures and your code. 
</font>
</p>



<p align="justify">
<font size="3">
As first step here is the to create a new RDD from the input text file './data/moby-dick.txt' that we already used in the first series of the exercises and that we saw again at the beginning of the lab. 
    
We can reuse the code and recall the presence of the function (already defined) that allows to remove non-letter characters.
</font>
</p>
<hr style="border:solid 2px;">

In [ ]:

'''
import re

def load_text(filename):
    return sc.textFile(filename)

Removes any non-letter character from the given word.

INPUT:
        word: A word

OUTPUT:
        the input word without the non-letter characters.

def remove_non_letters(word):
    return regex.sub('', word)

'''

mobydick = load_text('moby-dick.txt')
print("moby-dick text re-loaded in RDD and accessible in \"mobydick\" variable")

## 2.1 - Bigrams

<p align="justify">
<font size="3">
Describe Map Reduce procedure that gives as output the number of different bigrams that appear all along the document (if the bigram "is a" appears twice in the document must be counted just one time).
</font>
</p>

<p align="justify">
<font size="3">
In the following a function that parses a line of the text and produces the set of bigrams fot the is provided. 
Give a look to this function, we will se it in detail later.</font>
</p>

In [ ]:
from operator import add

'''
Returns the list of bigrams in a given text for each line (the end of a bigram search space is a line
of text:
"This  sentence
contains two different lines for looking for bigrams."
INPUT:
        a line of a text

OUTPUT:
        a list of bigrams for the text of the form b1_b2 where "b1_b2" is a string.

'''

def parse_bigrams(line):
    bigrams = []
    words = line.strip().split(" ")
    for i in range(len(words) - 1):
        bigram_w1 = remove_non_letters(words[i].lower()) 
        bigram_w2 = remove_non_letters(words[i+1].lower())
        if (len(bigram_w1) > 0) & (len(bigram_w2) > 0) :
                bigrams.append(bigram_w1 + "_" + bigram_w2)
    return bigrams

<p align="justify">
<font size="3">
Propose a Spark implementation of your procedure that follows a map-reduce approach.
</font>
</p>

In [ ]:
'''
Write here a procedure that given an RDD containing strings representing words returns the number of distinct bigrams.

INPUT:
        an RDD containing strings representing words

OUTPUT:
        the number of distinct bigrams.

'''

'''############## WRITE YOUR CODE HERE ##############'''

counts = 
output = counts.collect()
    
    
'''############## END OF THE EXERCISE ##############'''
 


print(counts.take(5))
print(counts.count())

################# EXPECTED OUTPUT #################
#
# [('call_me', 3), ('me_ishmael', 1), ('some_years', 1), ('years_agonever', 1), ('agonever_mind', 1)]
# 100513
###################################################

## 2.1 - Bigrams

<p align="justify">
<font size="3">
Sometimes when you write functions you do not realize that you can use a map-reduce series of steps.
Look at the provided function that parses bigrams and outputs a string representation of them. Re-think the functions using map, reduce, and filter operators,
    in Spark.
    </font>
</p>

<p align="justify">
<font size="3">
Propose a Spark implementation of the procedure  that follows a map-reduce approach, parses lines of text, and 
gives as output pairs of strings representing bigrams. For this step you just can to re-use the
 functions we already studied, for example:
<ul>
    <li> map() </li>
    <li> flatMap() </li>
    <li> filter() </li>
</ul>

Notice that you can be more flexible in the structure of the keys.
</font>
</p>

In [ ]:
'''
Returns a list of pairs of strings representing bigrams in a given text.
The procedure returns bigrams for each line (the search space of the bigrams is a line of text):
"This  sentence
contains two different lines and ('sentence','contains') is not in the output."
INPUT:
        an RDD containing a line of a text

OUTPUT:
        a list of pairs bigrams for the text of the form (b1, b2) where ('b1', 'b2')" is a pair of strings.

'''



'''############## WRITE YOUR CODE HERE ##############'''

def parse_smart_parse_bigrams(text_rdd):
    bigrams = 
    return bigrams
'''############## END OF THE EXERCISE ##############'''



bigrams=parse_smart_parse_bigrams(mobydick)
result = bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

print(result.count())
print(result.take(5))

################# EXPECTED OUTPUT #################
#
#  100513
# [(('me', 'ishmael'), 1), (('preciselyhaving', 'little'), 1), (('little', 'or'), 6), (('no', 'money'), 1), (('money', 'in'), 3)]
###################################################

## 2.2 - Bigrams search

<p align="justify">
<font size="3">
Look at the two procedures that parse bigrams and provide the output in two different format: is the type of the
   key influencing the map-reduce procedure? </font>
</p>

<p align="justify">
<font size="3">
From the performances point of view which data representation is better according to your opinion? Explain and motivate your answer
</font>
</p>

## 2.3 - Bigrams

<p align="justify">
<font size="3">
Now that we start being confident with bigrams and Map Reduce, we can start analysing the bigrams and write a procedure that gives as output the top 5 different bigrams that appear all along the document.
</font>
</p>

<p align="justify">
<font size="3">
Propose a Spark implementation of your procedure that transfomrs the key-value pairs and uses the top operation
</font>
</p>

In [ ]:
'''############## WRITE YOUR CODE HERE ##############'''
inverted_result = 
'''############## END OF THE EXERCISE ##############'''

print(inverted_result.top(5))




################# EXPECTED OUTPUT #################
#
# [(1726, ('of', 'the')), (1078, ('in', 'the')), (674, ('to', 'the')), (404, ('from', 'the')), (348, ('and', 'the'))]
###################################################

## 2.3 - Unique bigrams

<p align="justify">
<font size="3">
Try to re-use some of the steps you implemented in the previous exercise and provide the functions that counts the bigrams that appear only once in the text (if the bigram "is a" appears twice in the text it must not be counted in the final result).
</font>
</p>

In [ ]:
'''############## WRITE YOUR CODE HERE ##############'''


unique_bigrams = 


'''############## END OF THE EXERCISE ##############'''


print(unique_bigrams.take(5))

################# EXPECTED OUTPUT #################
#
#[(1, ('me', 'ishmael')), (1, ('preciselyhaving', 'little')), (1, ('no', 'money')), (1, ('way', 'i')), (1, ('driving', 'off'))]###################################################

# Second Part


<p align="justify">
<font size="3">
In this set of exercises you will continue using Spark for implementing algorithms that apply computations on tables and matrices starting to go beyond the strict application of map reduce programming paradigm (and Philosophy).
    
You will be required to implement the matrix multiplication.

</font>
</p>



# 3. Matrix Representation

<p align="justify">
<font size="3">
**Please read carefully this section that presents how matrices will be represented in this assignment.**
</font>
</p>

<p align="justify">
<font size="3">
Our input matrices are stored 
in textual files
As an example, the file matrix-a.txt_ looks like as follows:
<p>
0 1 2 4<br>
1 2 3 10<br>
2 12 15 150<br>
</p>
</font>
</p>
<p>
<font size="3">
Each line is a row in a matrix $A$. The first number of the line is the 
row identifier (starting from 0), the subsequent values (separated by a whitespace)
are the elements in each column of the row. The matrix represented in this file is the 
following:
<p>
<center>
  $A= \begin{bmatrix}
    1 & 2 & 4   \\
    2 & 3 & 10  \\
    12 & 15 & 150
\end{bmatrix}$
</center>
</font>
</p>


<p>
<font size="3">
We provide the implementation of  basic functions to load a matrix from file, visualize it
and get attributes.
</font>
</p>

## 3.1 Function $loadMatrix$

<p>
<font size="3">
The function $loadMatrix()$ loads a matrix from a file.
It takes in the name of the file and returns an RDD containing the matrix.

Each element of an RDD matrix is a key-value pair, where the key is the coordinate (row identifier, column identifier) of an element, and the value is the element itself.
For instance, the RDD corresponding to the matrix $A$ is the following:
<p>
$( (0, 0), 1 ), ( (0, 1), 2 ), ( (0, 2), 4 ), ( (1, 0), 2 ), ( (1, 1), 3 ), ( (1, 2), 10 ), ( (2, 0), 12 ), ( (2, 1), 15 ), ( (2, 2), 150 ) $
</p>
</font>
</p>

## 3.2 Function $shape$

<p>
<font size="3">
The function $shape()$ takes in an RDD matrix and returns the size of the matrix as a pair $(nbRows, nbCols)$, where $nbRows$ (resp., $nbCols$) denotes the number of rows (resp., columns) of the matrix.
</font>
</p>

## 3.3 Function $collect$

<p>
<font size="3">
The function $collect()$ takes in an RDD matrix and returns a representation of the matrix as a Python list $L$. Each element of $L$ is itself a list that corresponds to a row in the matrix.
For instance, the output of the function $collect$ for the matrix $A$ is as follows:   

<p>

$[ [1, 2, 4], [2, 3, 10], [12, 15, 150] ]$

</p>

</font>
</p>


## 3.4 Function $nice$

<p>
<font size="3">
The function $nice()$ prints the matrix in a nice and readable way.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize the definition of the functions**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
'''
Loads a matrix from a file.
INPUT: 
     the name of the input file
OUTPUT:
     an RDD containing the matrix
'''
def loadMatrix(filename):
    # Load the file into an RDD matrix
    matrix = sc.textFile(filename)
    # Splits each line. Each element is a list [nbRow, e1, e2, ..., ej]
    matrix = matrix.map(lambda line : line.split(' '))
    # Convert each element to a number (the first is an integer, the others are float)
    matrix = matrix.map(lambda row: [int(row[0])] + [float(row[i]) for i in range(1, len(row))])
    # Get an RDD where each element is a key-value pair ((row, col), element)
    matrix = matrix.flatMap(lambda row: [((row[0], j-1), row[j]) for j in range(1, len(row))])
    return matrix

'''
Returns the number of rows and colums of the matrix
INPUT: 
    An RDD representing a matrix
OUTPUT: 
    the size of the matrix as (nbRows, nbCols)
'''
def shape(matrix):
    M = collect(matrix)
    if len(M) == 0:
        return (0, 0)
    else:
        return (len(M), len(M[0]))

'''
Returns a matrix represented as a list of lists.
INPUT: 
    an RDD representing a matrix
OUTPUT: 
    the matrix represented as a list of lists.
'''
def collect(matrix):
    # Obtain an RDD, where the key is the row identifier and the value is (colId, element)
    matrix = matrix.map(lambda x: (x[0][0], (x[0][1], x[1])))
    # Groups all the values in a row.
    matrix = matrix.groupByKey()
    # Sorts the element by row identifier.
    matrix = matrix.sortByKey()
    # Sort the elements by column identifier.
    matrix = matrix.map(lambda x: sorted(list(x[1])))
    # Now obtain an RDD, where each element is a list containing the elements of a row.
    matrix = matrix.map(lambda row: [x[1] for x in row])
    # Finally, return the RDD as a Python list.
    return matrix.collect()
    
'''
Prints the matrix in a nice way.
INPUT: 
    the name of the matrix (var) and the matrix in the form of an RDD.
OUTPUT:
    - no output- it simply prints (shows) the matrix representation of the input
'''
def nice(var, matrix):
    # Obtain a representation of the matrix as a Python list.
    M = collect(matrix)
    # Print the name of the matrix
    print("Matrix ", var)
    # Print the matrix and format the output nicely
    print('\n'.join([''.join(['{:12.2f}'.format(item) for item in row]) 
      for row in M]))


# 4. Matrix Addition

<p align="justify">
<font size="3">
The code below loads two matrices $A$ and $B$ from file and calls the function $sum()$ to compute $A+B$.
</font>
</p>

<p>
<font size="3">
The function $sum()$ takes in:
<ul>
<li> $A$: an RDD containing the first matrix.
<li> $B$: an RDD containing the second matrix.
</ul>
The function $sum()$ returns an RDD containing the matrix obtained by summing $A$ and $B$.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $sum()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:
'''
Computes the sum of two matrices.
INPUT: 
    two RDDs containing the input matrices
OUTPUT: 
    the RDD containing the sum of the two input matrices
'''
        
def sum(A, B):
    '''############## WRITE YOUR CODE HERE ##############'''
    '''############### AND COMPLETE FOLLOWING THE INSTRUCTIONS ##############'''
    
    # Each element of the RDD A and B is ((r,c), e), where e is an element of the matrix and (r, c) is the 
    # coordinate of the element in terms of row and column.
    
    # 1. Put the two RDDs A and B together. Use the transformation union. Remember that a transformation 
    # always returns a new RDD with the result of the transformation.
    C = 
    
    #2. Transforms the RDD C into one where the values having the same key (i.e., same row and column) 
    # are summed together. Which transformation are you going to use on C?
    C =
    # We return the RDD containing the sum of the two input matrices
    return C

'''############## END OF THE EXERCISE ##############'''


# Load matrix A from file and print it.
A = loadMatrix("matrix-a.txt")
nice("A", A)

# Load matrix B from file and print it.
B = loadMatrix("matrix-b.txt")
nice("B", B)

# Compute A+B and print it
C = sum(A, B)
nice("C", C)

############################################################## 
#YOU SHOULD OBTAIN THE FOLLOWING MATRIX C AS RESULT
# 5.00        4.00        6.00      324.00       23.00
# 3.00        6.00       13.00      333.00      423.00
# 35.00       49.00      162.00       12.00        0.00
##############################################################


# 5. Scalar Multiplication

<p>
<font size="3">
The code below calls the function $scalarMultiply()$ to obtain the matrix $c\times A$, where $c$ is a scalar value.    
</font>
</p>

<p>
<font size="3">
The function $scalarMultiply()$ takes in:
<ul>
<li> $c$: a scalar value.
<li> $M$: an RDD containing a matrix.
</ul>
The function $scalarMultiply()$ returns an RDD containing the matrix obtained by multiplying $c$ with the input matrix.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $scalarMultiply()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:
'''
Computes the scalar multiplication.
INPUT: 
    a scalar value c and an RDD matrix M
OUTPUT:
    the RDD containing the matrix resulting from the scalar multiplication c * M.
'''
def scalarMultiply(c, M):
    '''############## WRITE YOUR CODE HERE ##############'''
    '''############### AND COMPLETE FOLLOWING THE INSTRUCTIONS ##############'''    
    # Apply a transformation on M, so each element of the matrix M is multiplied by c
    # Which transformation are you going to use?
    R = 
    
    return R
'''############## END OF THE EXERCISE ##############'''



# Prints 
nice("A", A)
nice("2*A", scalarMultiply(2, A))

############################################################## 
# THE RESULT SHOULD BE 
#2.00        4.00        8.00
#4.00        6.00       20.00
#24.00       30.00      300.00
##############################################################

# 6. Matrix Multiplication


<p>
<font size="3">
We want to implement a function $multiply()$ to obtain the matrix $A \times B$.
</font>
</p>

<p>
<font size="3">
The function $multiply()$ takes in:
<ul>
<li> $A$: an RDD containing the first matrix.
<li> $B$: an RDD containing the second matrix.
</ul>
The function $multiply$ returns an RDD containing the matrix obtained by multiplying the first and the second matrix.
The multiplication can only be computed if the number of columns of $A$ equals the number of rows of $B$.
</font>
</p>


<p>
<font size="3">
Let $A$ be an $n \times m$ matrix and $B$ an $m \times p$ matrix.
The matrix $C = A \times B$ is a $n \times p$ matrix, where each element $c_{i, k}$ is computed as 
follows:
<center>
  $c_{i, k} = \sum\limits_{j=0}^{m-1} a_{i, j} \cdot b_{j, k} \quad\quad (1)$ 
</center>
</font>
</p>


<p>
<font size="3">
One possible implementation of this function is based on a MapReduce schema.
Remember that in a MapReduce schema, the idea is to group elements by a key and apply a function to the elements 
that share the same key.
As you can see, for a given $(i, k)$, the element of $A$ that is in the j-th column is multiplied by the value of $B$ that is in the j-th row, for any column $j$.
Therefore, we can change the representation of $A$ and $B$ so that their elements are indexed by using $j$ as the key.
</font>
</p>

<p>
<font size="3">
More specifically, we can represent the matrix $A$ as follows:
<center>    
    $(j, (0, i, a_{i, j})) \quad 0 \leq i \leq n-1 \quad 0 \leq j \leq m-1  \quad\quad (2)$
</center>    
where the value $0$ in the triple $(0, i, a_{i, j})$ means that the element $a_{i, j}$ comes from the matrix $A$.
</font>
</p>


<p>
<font size="3">
Similarly, we can represent the matrix $B$ as follows:
<center>    
        $(j, (1, k, b_{j, k})) \quad 0 \leq j \leq m-1 \quad 0 \leq k \leq p-1 \quad\quad (3) $

</center>    
where the value $1$ in the triple $(1, k, b_{j, k})$ means that the element $b_{j, k}$ comes from the matrix $B$.
</font>
</p>

<p>
<font size="3">
As a first step, we want to code two functions $transformA()$ and $transformB()$ to obtain the two representations of $A$ and $B$ respectively, as described in Equation (2) and (3).
The two representations returned by both functions **must be RDDs**.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $transformA()$ and $transformB()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:

'''
Transforms the RDD matrix A into an RDD as described in Equation (2)
INPUT:
    an RDD that contains data representing a matrix 
OUTPUT:
    a matrix representation according to the Equation (2)
'''

def transformA(A):
    '''############## WRITE YOUR CODE HERE ##############'''
    # Transform A as indicated in the text above
     A = 
    '''############## END OF THE FIRST PART OF THE EXERCISE ##############'''
    return A


'''
Transforms the RDD matrix B into an RDD as described in Equation (3)
INPUT:
    an RDD that contains data representing a matrix 
OUTPUT:
    a matrix representation according to the Equation (3)
'''
def transformB(B):
    '''############## WRITE YOUR CODE HERE ##############'''
    # Transform B as indicated in the text above
    B = 
    '''############## END OF THE EXERCISE ##############'''
    return B


# Displayes the two matrices
nice("A", A)
nice("B", B)

# Transforms them
Atransformed = transformA(A)
Btransformed = transformB(B)

# Display the result.
print("\n********** Representation for A ************\n")
print(Atransformed.collect())
print("\n********** Representation for B ************\n")
print(Btransformed.collect())


<p>
<font size="3">
In order to group all the elements of both matrices by the key $j$, we need to merge the two RDDs $Atransformed$ and $Btransformed$.
The function $merge()$ declared below takes in $Atransformed$ and $Btransformed$ and returns an RDD that results from the union of the two input RDDs.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $merge()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:
'''
Returns the union of Atransformed and Btransformed.
INPUT:
    two matrix representation following the Equation (2) and Equation (3) representation
OUTPUT:
    the union of Atransformed and Btransformed.
'''
def merge(Atransformed, Btransformed):
    '''############## WRITE YOUR CODE HERE ##############'''
    # Put together the two input RDDs
    R = 
    '''############## END OF THE EXERCISE ##############'''

    return R

nice("A", A)
nice("B", B)
    
merged = merge(Atransformed, Btransformed)    

print("\n********** Representation for merged ************\n")
print(merged.collect())



<p>
<font size="3">
Now we can group the values of the RDD $merged$ obtained above by their key $j$. 
We define a function $group()$ that returns an RDD obtained by grouping the values of the input RDD by their key.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $group()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:
'''
Returns an RDD where the values of the input RDD are grouped by their key.
INPUT:
    An RDD
OUTPUT:
    The values of the RDD grouped by key
'''
def group(merged):
    '''############## WRITE YOUR CODE HERE ##############'''
    # Groups the element of the input RDD by key.
    R = 
    '''############## END OF THE EXERCISE ##############'''
    return R

nice("A", A)
nice("B", B)
    
grouped = group(merged)    

print("\n********** Representation for grouped ************\n")
L = grouped.collect()
print('[')
for l in L:
    print("(",l[0], ",", end='', sep='')
    print("[", end='')
    for el in l[1]:
        print(el, end="")
    print("],")
print(']')

######################################################################
# Note that in the output, each element is (j, L), where
# L is a list that contains all the elements in the j-th column of A
# and all the elements in j-th row of B
######################################################################

<p>
<font size="3">
Each element of the RDD $grouped$ obtained above is a key-value pair, where the key is the index $j$ and the value is a list $L$ containing all the triples corresponding to the elements of matrix $A$ in the $j-$th column  and the elements of matrix $B$  in the $j-$row, as follows: 
<p>
<center>
$(0, i, a_{i, j})\ 0 \leq i \leq n-1 \quad (1, k, b_{j, k})\ 0 \leq k \leq p-1$
</center>
</p>
<p>
Remember that all triples corresponding to matrix $A$ have 0 as their first value, while those corresponding to matrix $B$ have 1.
</p>
<p>
From Equation (1), you can see that the product $a_{i, j} \cdot b_{j, k}$ contributes to the value $c_{i, k}$, for $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$. 
Therefore, given the list $L$, we associate each value $a_{i, j} \cdot b_{j, k}$  to the pair $(i, k)$.
</p>
<p>
In other words, we now transform the RDD $grouped$ into an RDD where 
each element is a key-value pair, where the key is $(i, k)$ and the value is $a_{i, j} \cdot b_{j, k}$.

</p>    
<p>
In the code below, the function $multiplyElements()$ is given that takes in a value $(j, L)$ of the RDD $grouped$ 
and returns a list, where each element is a pair $((i, k), a_{i, j} \cdot b_{j, k})$, $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$.  
</p>
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $groupProducts()$ that:
    <ul>
      <li> Takes in the RDD $grouped$.
      <li> Applies the function $multiplyElements()$ to each element of $grouped$.
      <li> Returns an RDD where each element is $((i, k), a_{i, j} \cdot b_{j, k})$, $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$. 
    </ul>
    Execute the code.**</font>
<hr style=" border:none; height:2px;">
</p>




In [ ]:
import sys

'''
Multiplies each element from matrix A with each element from 
matrix B in the list L (see description above).
INPUT: 
    a value (j, L) of the RDD grouped.
OUTPUT:
    a list where each element is ((i, k), a_ij * b_jk)
'''
def multiplyElements(value):
    j = value[0]
    L = value[1]
    
    '''
    The output key-value pairs.
    '''
    kv = []
    '''
    Maybe not necessary, we make sure that all triples with 
    the first element 0 (those from matrix A)
    comes before any triple from matrix B.
    '''
    L = sorted(list(L))
    
    '''
    For convenience, we store the triples from matrix A
    and those from matrix B in two separate lists 
    LA and LB.
    '''
    sep = 0
    while L[sep][0] == 0:
        sep += 1
    LA = [L[i] for i in range(0, sep)]
    LB = [L[i] for i in range(sep, len(L))]
    '''
    For each element (0, i, a_ij) in LA  
    and each element (1, k, b_jk) in LB, 
    we add the pair ((i, k), a_ij * b_jk) to kv.
    '''
    for a in LA:
        for b in LB:
            i = a[1]
            k = b[1]
            kv.append(((i, k), a[2]*b[2]))
    return kv

'''
Returns an RDD where each value is a pair ((i, k), a_ij * b_jk)
INPUT:
    an RDD whose elements are grouped by key
OUTPUT:
     an RDD where each value is a pair ((i, k), a_ij * b_jk)
'''
def groupProducts(grouped):
    '''############## WRITE YOUR CODE HERE ##############'''
    # Apply a transformation to the input RDD to get an RDD as described in the text.
    # Which tranformations are you going to use? map or flatMap?
    R = 
    '''############## END OF THE EXERCISE ##############'''
    return R



nice("A", A)
nice("B", B)

print("\n********** Representation for multipliedElements ************\n")
multipliedElements = groupProducts(grouped)
print(multipliedElements.collect())

<p>
<font size="3">
Each element of the RDD $multipliedElements$ obtained above is $((i, k), a_{i, j} \cdot b_{j, k})$, 
$0 \leq i \leq n-1$, $0 \leq j \leq m-1$, $0 \leq k \leq p-1$. 
From Equation (1), we can see that each $c_{i, k}$ is obtained by summing up the products $a_{i, j} \cdot b_{j, k}$, $0 \leq j \leq m-1$.
<p>
Therefore, in order to obtain the matrix $C = A \times B$, the only thing that we need to do is to sum all 
values $a_{i, j} \cdot b_{j, k}$ associated with the same key $(i, k)$.
</p>
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $getResult()$ that:
    <ul>
      <li> Takes in the RDD $multipliedElements$.
      <li> Transforms the input RDD into one where each element is $((i, k), \sum\limits_{j=0}^{m-1} a_{i, j} \cdot b_{j, k})$
      <li> Returns the resulting RDD. 
    </ul>
    Execute the code.We finally obtain the matrix $C$.**</font>
<hr style=" border:none; height:2px;">
</p>



In [ ]:
'''
Returns an RDD where all the values with the same keys are summed.
INPUT:
    an RDD 
OUTPUT:
     an RDD where all the values with the same keys are summed.
'''
def getResult(multipliedElements
    '''############## WRITE YOUR CODE HERE ##############'''
    # Apply a transformation to the input RDD so that all values with the same key are summed.
    R = 
    '''############## END OF THE EXERCISE ##############'''
    return R


nice("A", A)
nice("B", B)
C = getResult(multipliedElements)
nice("C", C)

############################################################## 
#YOU SHOULD OBTAIN THE FOLLOWING MATRIX C AS RESULT
# 98.00      144.00       56.00     1038.00      869.00
# 241.00      353.00      133.00     1767.00     1315.00
# 3513.00     5169.00     1869.00    10683.00     6621.00
##############################################################


<p>
<font size="3">
We now wrap every function that we implemented above in only one function $multiply()$ that we can use any time we need to multiply two matrices.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the code below to define the function $multiply()$**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
def multiply(A, B):
  # lambda ((i, j), v): (j, (i, v))
  left = A.map(lambda e: (e[0][1], (e[0][0], e[1])))
  # lambda ((j, k), w): (j, (k, w))
  right = B.map(lambda e: (e[0][0], (e[0][1], e[1])))
  productEntries = left.join(right)
  # lambda (x, ((i, v), (k, w))): ((i, k), (v * w))
  productEntries = productEntries.map(lambda e: ( (e[1][0][0], e[1][1][0]), (e[1][0][1] * e[1][1][1]) ) )\
                  .reduceByKey(lambda x,y: x+y)
  return productEntries